<a href="https://colab.research.google.com/github/jdeiros/soa-2020/blob/master/HPC/Deiros_Jeronimo_ejercicio_1_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción

Por lo general, las computadoras se utilizan para compilar y analizar los resultados de encuestas y estudios de opinión.

El siguiente cuaderno calcula la **moda** de los **N** valores de los elementos de un vector (vector_resultados), lo hace en forma paralela utilizando GPGPU. Cada elemento del vector se inicializa con resultados aleatorios de puntajes (numeros enteros entre 0 y 9).

El algoritmo se basa en un ejemplo práctico del libro "C/C++ Cómo Programar"[1] 

Su objetivo es aprender a utilizar Python[2] en la plataforma Colab [3] y CUDA[5,6].  Mostrando el funcionamiento y granularidad (grilla, bloque, warps) de sobre una dimensión (x).

#2. Armado de ambiente

Instalación de modulo CUDA  de Python.

In [1]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 7.5MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621008 sha256=6ad3d8c17dbcd2bee2c64dd6a373a0967d5cc3648e5e7b8bf810aa74411035bd
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=9b68de26d815c1e29d0f3bbf3d3cea3780ca48503e906dc50f102eed9eae627c
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3. Desarrollo


In [12]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidad_elementos =   500000#@param {type: "number"}
# --------------------------------------------

import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
from datetime import datetime
from   pycuda.compiler import SourceModule

tiempo_total = datetime.now()

# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) \
              * 1000 + dt.microseconds / 1000.0
try:
    # CPU: Inicializo el vector resultados con puntajes (de 0 a 9) aleatorios
    # con cantidad de elementos ingresada.
    vector_resultados_cpu = np.random.randint(0, 10, size = cantidad_elementos)
    vector_resultados_cpu = np.array(vector_resultados_cpu, dtype=np.int32)
    
    # vector_resultados_cpu.astype(np.int32())

    # Inicializo en cero el vector frecuencia (puntajes de 0 a 9)
    vector_frecuencias_cpu = [0 for i in range(10)]
    vector_frecuencias_cpu = np.array(vector_frecuencias_cpu, dtype=np.int32)
    # vector_frecuencias_cpu.astype( numpy.int32() )
    print("-------------------------------------------------")
    print("vector respuestas cpu:")
    print(vector_resultados_cpu)
    

    # CPU - reservo la memoria GPU.
    vector_resultados_gpu = cuda.mem_alloc(vector_resultados_cpu.nbytes)
    vector_frecuencias_gpu = cuda.mem_alloc(vector_frecuencias_cpu.nbytes)

    # GPU - Copio la memoria al GPU.
    cuda.memcpy_htod(vector_resultados_gpu, vector_resultados_cpu)
    cuda.memcpy_htod(vector_frecuencias_gpu, vector_frecuencias_cpu)

    # CPU - Defino la función kernel que ejecutará en GPU.
    module = SourceModule("""
    __global__ void kernel_encuesta(int n, int *frecuencias , int *resultados)
    {
        int idx = threadIdx.x + blockIdx.x*blockDim.x;

        if(idx < n)
        {
            ++frecuencias[resultados[idx]];        
        }
    }
    """)

    # CPU - Genero la función kernel.
    kernel = module.get_function("kernel_encuesta")
    
    tiempo_gpu = datetime.now()
    
    dim_hilo = 256
    dim_bloque = np.int( (cantidad_elementos + dim_hilo - 1) / dim_hilo )
    print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
    
    # GPU - Ejecuta el kernel.
    kernel( np.int32(cantidad_elementos), \
            vector_frecuencias_gpu, \
            vector_resultados_gpu, \
            block=( dim_hilo, 1, 1 ), grid=(dim_bloque, 1,1) )

    tiempo_gpu = datetime.now() - tiempo_gpu

    # GPU - Copio el resultado desde la memoria GPU.
    cuda.memcpy_dtoh( vector_frecuencias_cpu, vector_frecuencias_gpu )

    
    # CPU - Informo el resultado.
    print("-------------------------------------------------")
    print("vector frecuencia: ")
    print(vector_frecuencias_cpu)
        
    mas_grande = 0
    valor_moda = 0

    # obtengo la moda del vector_frecuencias_cpu
    for rango in range(0, len(vector_frecuencias_cpu)):
        if(vector_frecuencias_cpu[rango] > mas_grande):
            mas_grande = vector_frecuencias_cpu[rango]
            valor_moda = rango
    
    print("-------------------------------------------------")
    print(f"Para esta ejecución la moda es: {valor_moda}, ")
    print(f"cual ocurrió {mas_grande} veces.")

    tiempo_total = datetime.now() - tiempo_total
    print("-------------------------------------------------")
    print("Tiempo Total: ", tiempo_en_ms(tiempo_total),"[ms]")
    print("Tiempo gpu: ", tiempo_en_ms(tiempo_gpu), "[ms]" )

except Exception as e:
  print(f"error: {e}")



-------------------------------------------------
vector respuestas cpu:
[6 1 7 ... 2 6 9]
Thread x:  256 , Bloque x: 1954
-------------------------------------------------
vector frecuencia: 
[49 49 49 49 49 49 49 49 43 43]
-------------------------------------------------
Para esta ejecución la moda es: 0, 
cual ocurrió 49 veces.
-------------------------------------------------
Tiempo Total:  15.071 [ms]
Tiempo gpu:  0.4 [ms]


#4. Tabla de pasos

Paso | Procesador | Funcion | Detalle
------------ | ------------ | ------------- | -------------
1 | CPU | @param | Lectura del tamaño de vector de Colab.
2 | CPU | import | Importa los módulos para funcionar.
3 | CPU | datetime.now() | Toma el tiempo actual.
4 | CPU | np.random.randint(0, 10, size = cantidad_elementos) | Inicializa el vector _vector_resultados_ con puntajes (de 0 a 9) aleatorios en cada elemento, en una cantidad de elementos ingresada en el paso 1.
5 | **GPU** | cuda.mem_alloc()] | Reserva la memoria en GPU.
6 | **GPU** | cuda.memcpy_htod() | Copia las memorias desde el CPU al GPU.
7 | CPU | SourceModule() | Define el código del kernel.
8 | CPU | module.get_function() | Genera la función del kernel GPU
9 | CPU | dim_tx/dim_bx | Calcula las dimensiones.
10 | **GPU**  | kernel() | Ejecuta el kernel en GPU
11 | CPU | cuda.memcpy_dtoh( ) | Copia el resultado desde GPU memoria A a CPU memoria R.
12 | CPU | print() | Informo los resultados.

#5. Conclusiones

* ### 5.1 Breve repaso de los puntos mas relevantes del trabajo.
Se definió una función kernel para ser ejecutada en gpu a traves de la clase SourceModule() de cuda[5]. Dentro de la funcion kernel_encuesta() se calcularon las frecuencias de cada votación de una encuesta con puntajes de 0 a 9 representadas por el vector frecuencias. Si bien, el algoritmo no esta funcionando correctamente como se explica en la seccion 5.2 de lecciones aprendidas, se decide dejarlo como esta para evidenciar el error.
De todas formas, si observamos los tiempos de ejecución tomados, ya que los hilos se ejecutan por mas que no estan sumarizando correctamente los datos del vector frecuencias, podemos ver que hay una mejora significativa en el procesamiento comparado con la version secuencial que corre en cpu. Podemos ver que la mayor parte del tiempo se consume en el programa en general, mientras que el tiempo de procesamiento del gpu es muy inferior. Con lo cual, de funcionar correctamente, optimizaría su ejecución.

* ### 5.2 Explicación sobre las lecciones aprendidas que deja el ejercicio.
Como se puede observar, el resultado no es el esperado en el ejercicio. Vemos que el vector de frecuencias empieza a sumar las apariciones de los puntajes, recien cuando se carga una cantidad de elementos para las respuestas por encima de los 40k.
Por debajo de esa cantidad, veremos que la mayor frecuencia de cada elemento no supera la cantidad de 1 aparicion y esto no es lo que se ve en el vector de resultados, con lo cual se obtienen datos erroneos.
Esto puede deberse, en principio al uso del vector frecuencias. Este se pasa al device (gpu) a traves de la funcion de cuda memcpy_htod(), que genera una copia de la memoria principal a la memoria del dispositivo. En el se realiza un calculo que **no** es independiente del resto[4], dado que al sumar una aparicion, por ejemplo del numero 8 (elemento 8 del vector frecuencias), en varios hilos que estan corriendo en paralelo, solo se sumara una vez, es decir, se incrementa en uno el contador de frecuencias para el numero 8, cuando debería sumar todas las apariciones en cada uno de esos hilos que estan corriendo en paralelo. Sin embargo, al realizar pruebas con cantidades de elemntos superiores, mayores a 50k veremos que si se estan incrementando las frecuencias, en el orden de (cantidad_elementos / 10k), por ejemplo para 500k elementos, las apariciones aleatorias de cada puntaje en el vector de frecuencia ronda los 50. Con esta cantidad de elementos, al definir la cantidad de 256 hilos, los bloques que se necesitaron para resolver fueron de 1954 para llegar a cubrir los 500k. Evidentemente, la cantidad de hilos que acceden concurrentemente al vector frecuencias es de 10k aproximadamente.
Este algoritmo no esta funcionando correctamente, pero decidí dejarlo asi para poder evidenciar lo aprendido y, de alguna manera, ver las limitaciones o problemas con los que podemos enfrentarnos al trabajar con este mecanismo.

* ### 5.3 Sugerencias para continuar con el ejercicio (funcionalidad o algoritmo).
Poder solucionar este problema de concurrencia utilizando eficientemente los hilos. O plantear otra solucion cambiando el algoritmo.



#6. Bibliografía

* [1] Como Programar en C C++ y Java 4ta Edición Harvey M. Deitel & Paul J. Deitel.
* [2] Python Básico - SOA UNLaM: [Python Básico](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)
* [3] Tutorial Point Colab: [Google Colab Tutorial](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/google_colab_tutorial.pdf)
* [4] Computacion de alto desempeño en GPU: [pdf](http://so-unlam.com.ar/material-clase/HPC/Computaci%C3%B3n%20de%20alto%20desempe%C3%B1o%20en%20GPU.pdf)
* [5] Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)
* [6] Repositorio de PyCUDA: [WEB](https://pypi.org/project/pycuda/)